# 🚀 UIDAI Hackathon Dashboard - Insight Generator

**High-Performance Backend for Aadhaar Data Analysis**

This notebook processes Aadhaar Enrolment, Biometric, and Demographic data to generate actionable operational insights:
- **MBU Deficit Tracking** - Children enrolled but lacking biometric updates
- **Fraud Detection** - 4-Model Ensemble anomaly detection
- **Migration Hotspots** - Areas with high in-migration patterns

---

## 📦 Part 0: Imports & Configuration

In [ ]:
import glob
import json
import warnings
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✅ All imports successful!")

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

BASE_PATH = Path("./Data")

DATA_CONFIG = {
    'enrolment': {
        'folder': 'api_data_aadhar_enrolment',
        'pattern': '*_enrolment_*.csv',
        'columns': ['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater']
    },
    'biometric': {
        'folder': 'api_data_aadhar_biometric',
        'pattern': '*_biometric_*.csv',
        'columns': ['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_']
    },
    'demographic': {
        'folder': 'api_data_aadhar_demographic',
        'pattern': '*_demographic_*.csv',
        'columns': ['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17_']
    }
}

# Anomaly detection configuration
ANOMALY_CONFIG = {
    'contamination': 0.1,
    'random_state': 42,
    'autoencoder_threshold_percentile': 90,
    'mbu_deficit_threshold': 500,
    'fraud_vote_threshold': 3
}

print("✅ Configuration loaded!")
print(f"📁 Base Path: {BASE_PATH.absolute()}")

---
## 📊 Part 1: Data Loading & Preprocessing

In [ ]:
def load_csv_chunks(folder: str, pattern: str, columns: List[str]) -> pd.DataFrame:
    """
    Load all CSV chunks matching the pattern from the specified folder.
    Verifies file integrity and concatenates into a single DataFrame.
    """
    folder_path = BASE_PATH / folder
    files = glob.glob(str(folder_path / pattern))
    
    if not files:
        print(f"  ⚠️  No files found matching {pattern} in {folder_path}")
        return pd.DataFrame(columns=columns)
    
    print(f"  📂 Found {len(files)} file(s) in {folder}")
    
    dfs = []
    for file_path in sorted(files):
        try:
            df = pd.read_csv(file_path, usecols=lambda c: c in columns, low_memory=False)
            missing_cols = set(columns) - set(df.columns)
            if missing_cols:
                print(f"    ⚠️  Missing columns in {Path(file_path).name}: {missing_cols}")
                continue
            dfs.append(df)
            print(f"    ✓ Loaded {Path(file_path).name}: {len(df):,} rows")
        except Exception as e:
            print(f"    ✗ Error loading {Path(file_path).name}: {e}")
            continue
    
    if not dfs:
        return pd.DataFrame(columns=columns)
    
    return pd.concat(dfs, ignore_index=True)


def clean_pincode(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filter rows where pincode is numeric and exactly 6 digits.
    """
    df['pincode'] = df['pincode'].astype(str).str.strip()
    mask = df['pincode'].str.match(r'^\d{6}$', na=False)
    filtered_df = df[mask].copy()
    
    removed = len(df) - len(filtered_df)
    if removed > 0:
        print(f"    🔍 Filtered out {removed:,} rows with invalid pincodes")
    
    return filtered_df


def preprocess_dataframe(df: pd.DataFrame, name: str, numeric_cols: List[str]) -> pd.DataFrame:
    """
    Preprocess DataFrame: convert date, clean pincode, aggregate by location.
    """
    print(f"\n  🔧 Preprocessing {name}...")
    
    if df.empty:
        print(f"    ⚠️  Empty DataFrame, skipping preprocessing")
        return df
    
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')
    
    # Clean pincodes
    df = clean_pincode(df)
    
    if df.empty:
        return df
    
    # Ensure numeric columns are numeric
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    
    # Aggregate by pincode, district, state
    agg_dict = {col: 'sum' for col in numeric_cols if col in df.columns}
    df_agg = df.groupby(['pincode', 'district', 'state'], as_index=False).agg(agg_dict)
    
    print(f"    ✓ Aggregated to {len(df_agg):,} unique pincodes")
    
    return df_agg

print("✅ Helper functions defined!")

In [ ]:
# Load Enrolment Data
print("📁 Loading Enrolment Data...")
df_enrol = load_csv_chunks(
    DATA_CONFIG['enrolment']['folder'],
    DATA_CONFIG['enrolment']['pattern'],
    DATA_CONFIG['enrolment']['columns']
)
df_enrol = preprocess_dataframe(
    df_enrol, 
    'enrolment',
    ['age_0_5', 'age_5_17', 'age_18_greater']
)

In [ ]:
# Load Biometric Data
print("📁 Loading Biometric Data...")
df_bio = load_csv_chunks(
    DATA_CONFIG['biometric']['folder'],
    DATA_CONFIG['biometric']['pattern'],
    DATA_CONFIG['biometric']['columns']
)
df_bio = preprocess_dataframe(
    df_bio,
    'biometric',
    ['bio_age_5_17', 'bio_age_17_']
)

In [ ]:
# Load Demographic Data
print("📁 Loading Demographic Data...")
df_demo = load_csv_chunks(
    DATA_CONFIG['demographic']['folder'],
    DATA_CONFIG['demographic']['pattern'],
    DATA_CONFIG['demographic']['columns']
)
df_demo = preprocess_dataframe(
    df_demo,
    'demographic',
    ['demo_age_5_17', 'demo_age_17_']
)

In [ ]:
# Summary
print("\n" + "="*50)
print("✅ DATA LOADING COMPLETE!")
print("="*50)
print(f"   Enrolment:   {len(df_enrol):,} pincodes")
print(f"   Biometric:   {len(df_bio):,} pincodes")
print(f"   Demographic: {len(df_demo):,} pincodes")

In [ ]:
# Preview data
print("📋 Enrolment Data Sample:")
df_enrol.head()

---
## 🔬 Part 2: Anomaly Detection Engine (4-Model Ensemble)

In [ ]:
def prepare_features(df_enrol: pd.DataFrame, df_bio: pd.DataFrame, df_demo: pd.DataFrame) -> Tuple[pd.DataFrame, np.ndarray, StandardScaler]:
    """
    Create feature DataFrame indexed by pincode for anomaly detection.
    """
    print("📐 Preparing features for ensemble models...")
    
    # Start with enrolment
    df_features = df_enrol[['pincode', 'district', 'state', 'age_18_greater']].copy()
    df_features = df_features.set_index('pincode')
    
    # Merge biometric data
    if not df_bio.empty:
        df_bio_indexed = df_bio[['pincode', 'bio_age_5_17', 'bio_age_17_']].set_index('pincode')
        df_features = df_features.join(df_bio_indexed, how='left')
    else:
        df_features['bio_age_5_17'] = 0
        df_features['bio_age_17_'] = 0
    
    # Merge demographic data
    if not df_demo.empty:
        df_demo_indexed = df_demo[['pincode', 'demo_age_5_17', 'demo_age_17_']].set_index('pincode')
        df_features = df_features.join(df_demo_indexed, how='left')
    else:
        df_features['demo_age_5_17'] = 0
        df_features['demo_age_17_'] = 0
    
    df_features = df_features.fillna(0)
    
    # Feature columns for anomaly detection
    feature_cols = ['age_18_greater', 'bio_age_17_', 'demo_age_17_']
    X = df_features[feature_cols].values
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(f"✓ Created feature matrix: {X_scaled.shape[0]} samples, {X_scaled.shape[1]} features")
    
    return df_features, X_scaled, scaler

df_features, X_scaled, scaler = prepare_features(df_enrol, df_bio, df_demo)

In [ ]:
# Model 1: Isolation Forest
print("🌲 Running Isolation Forest...")
clf_if = IsolationForest(contamination=0.1, random_state=42, n_jobs=-1)
pred_if = clf_if.fit_predict(X_scaled)
votes_if = (pred_if == -1).astype(int)
print(f"   ✓ Detected {votes_if.sum():,} anomalies ({100*votes_if.mean():.1f}%)")

In [ ]:
# Model 2: One-Class SVM
print("🎯 Running One-Class SVM...")
clf_svm = OneClassSVM(nu=0.1, kernel='rbf', gamma='scale')
pred_svm = clf_svm.fit_predict(X_scaled)
votes_svm = (pred_svm == -1).astype(int)
print(f"   ✓ Detected {votes_svm.sum():,} anomalies ({100*votes_svm.mean():.1f}%)")

In [ ]:
# Model 3: Local Outlier Factor
print("📍 Running Local Outlier Factor...")
clf_lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1, n_jobs=-1)
pred_lof = clf_lof.fit_predict(X_scaled)
votes_lof = (pred_lof == -1).astype(int)
print(f"   ✓ Detected {votes_lof.sum():,} anomalies ({100*votes_lof.mean():.1f}%)")

In [ ]:
# Model 4: Autoencoder (Deep Learning)
print("🧠 Running Autoencoder (Deep Learning)...")

votes_ae = None
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    
    tf.get_logger().setLevel('ERROR')
    
    # Build Autoencoder
    input_dim = X_scaled.shape[1]
    inputs = keras.Input(shape=(input_dim,))
    x = layers.Dense(8, activation='relu')(inputs)
    x = layers.Dense(4, activation='relu')(x)  # Bottleneck
    x = layers.Dense(8, activation='relu')(x)
    outputs = layers.Dense(input_dim, activation='linear')(x)
    
    autoencoder = keras.Model(inputs, outputs)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    # Train
    autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=256, shuffle=True, verbose=0, validation_split=0.1)
    
    # Calculate reconstruction error
    reconstructions = autoencoder.predict(X_scaled, verbose=0)
    mse = np.mean(np.power(X_scaled - reconstructions, 2), axis=1)
    threshold = np.percentile(mse, 90)
    votes_ae = (mse > threshold).astype(int)
    
    print(f"   ✓ Detected {votes_ae.sum():,} anomalies ({100*votes_ae.mean():.1f}%)")
    print(f"   ✓ MSE Threshold: {threshold:.4f}")
    
except ImportError:
    print("   ⚠️ TensorFlow not available. Using 3-model voting.")
except Exception as e:
    print(f"   ⚠️ Autoencoder failed: {e}")

In [ ]:
# Ensemble Voting
print("\n" + "="*50)
print("📊 ENSEMBLE VOTING")
print("="*50)

if votes_ae is not None:
    total_votes = votes_if + votes_svm + votes_lof + votes_ae
    total_models = 4
    print("Using 4 models (including Autoencoder)")
else:
    total_votes = votes_if + votes_svm + votes_lof
    total_models = 3
    print("Using 3 models (Autoencoder unavailable)")

# Add results to DataFrame
df_results = df_features.copy()
df_results['votes'] = total_votes
df_results['total_models'] = total_models
df_results['confidence_score'] = (total_votes / total_models) * 100
df_results['is_high_risk'] = total_votes >= ANOMALY_CONFIG['fraud_vote_threshold']

high_risk_count = df_results['is_high_risk'].sum()
print(f"\n🚨 HIGH RISK AREAS (votes >= {ANOMALY_CONFIG['fraud_vote_threshold']}): {high_risk_count:,}")

In [ ]:
# View high-risk pincodes
print("🔴 Top 10 High-Risk Pincodes:")
df_results[df_results['is_high_risk']].sort_values('confidence_score', ascending=False).head(10)

---
## 📝 Part 3: Business Logic - Ticket Generation

In [ ]:
def generate_mbu_compliance_tickets(df_enrol: pd.DataFrame, df_bio: pd.DataFrame) -> List[Dict[str, Any]]:
    """
    Generate MBU compliance tickets.
    Deficit = Enrolment[age_5_17] - Biometric[bio_age_5_17]
    """
    print("📋 Generating MBU Compliance Tickets...")
    
    tickets = []
    df_merged = df_enrol[['pincode', 'district', 'state', 'age_5_17']].merge(
        df_bio[['pincode', 'bio_age_5_17']], on='pincode', how='left'
    )
    df_merged['bio_age_5_17'] = df_merged['bio_age_5_17'].fillna(0)
    df_merged['deficit'] = df_merged['age_5_17'] - df_merged['bio_age_5_17']
    
    threshold = ANOMALY_CONFIG['mbu_deficit_threshold']
    df_high_deficit = df_merged[df_merged['deficit'] > threshold]
    
    for _, row in df_high_deficit.iterrows():
        ticket = {
            'pincode': row['pincode'],
            'district': row['district'],
            'state': row['state'],
            'priority': 'High',
            'task': 'MBU Camp',
            'venue': 'Schools',
            'details': f"{int(row['deficit'])} kids pending biometric update",
            'deficit': int(row['deficit']),
            'whatsapp_msg': f"🏫 MBU CAMP NEEDED\n📍 {row['district']}, {row['state']}\n📮 Pincode: {row['pincode']}\n👦 {int(row['deficit'])} children pending\n⚠️ Priority: HIGH"
        }
        tickets.append(ticket)
    
    print(f"   ✓ Generated {len(tickets):,} MBU compliance tickets")
    return tickets


def generate_fraud_alert_tickets(df_results: pd.DataFrame) -> List[Dict[str, Any]]:
    """
    Generate Fraud Alert tickets from ensemble anomaly detection.
    """
    print("🚨 Generating Fraud Alert Tickets...")
    
    tickets = []
    df_high_risk = df_results[df_results['is_high_risk']].reset_index()
    
    for _, row in df_high_risk.iterrows():
        ticket = {
            'pincode': row['pincode'],
            'district': row['district'],
            'state': row['state'],
            'priority': 'Critical',
            'task': 'Fraud Audit',
            'venue': 'CSC Center',
            'details': f"Suspicious Adult Enrolments (Confidence: {row['confidence_score']:.0f}%)",
            'votes': int(row['votes']),
            'confidence': row['confidence_score'],
            'whatsapp_msg': f"🚨 FRAUD ALERT\n📍 {row['district']}, {row['state']}\n📮 Pincode: {row['pincode']}\n🎯 Confidence: {row['confidence_score']:.0f}%\n⚠️ Priority: CRITICAL"
        }
        tickets.append(ticket)
    
    print(f"   ✓ Generated {len(tickets):,} fraud alert tickets")
    return tickets


def generate_migration_hotspot_tickets(df_enrol: pd.DataFrame, df_demo: pd.DataFrame) -> List[Dict[str, Any]]:
    """
    Generate Migration Hotspot tickets.
    Condition: demo_age_17_ in top 10% AND age_0_5 in bottom 20%
    """
    print("🏙️ Generating Migration Hotspot Tickets...")
    
    tickets = []
    df_merged = df_enrol[['pincode', 'district', 'state', 'age_0_5']].merge(
        df_demo[['pincode', 'demo_age_17_']], on='pincode', how='left'
    )
    df_merged['demo_age_17_'] = df_merged['demo_age_17_'].fillna(0)
    
    demo_top_10_threshold = df_merged['demo_age_17_'].quantile(0.90)
    age_0_5_bottom_20_threshold = df_merged['age_0_5'].quantile(0.20)
    
    mask = (
        (df_merged['demo_age_17_'] >= demo_top_10_threshold) & 
        (df_merged['age_0_5'] <= age_0_5_bottom_20_threshold)
    )
    df_hotspots = df_merged[mask]
    
    for _, row in df_hotspots.iterrows():
        ticket = {
            'pincode': row['pincode'],
            'district': row['district'],
            'state': row['state'],
            'priority': 'Medium',
            'task': 'Urban Planning Survey',
            'venue': 'Municipal Ward',
            'details': 'High In-Migration Detected',
            'whatsapp_msg': f"📊 MIGRATION ALERT\n📍 {row['district']}, {row['state']}\n📮 Pincode: {row['pincode']}\n📋 Action: Urban Planning Survey"
        }
        tickets.append(ticket)
    
    print(f"   ✓ Generated {len(tickets):,} migration hotspot tickets")
    return tickets

print("✅ Ticket generation functions defined!")

In [ ]:
# Generate all tickets
print("\n" + "="*50)
print("📝 GENERATING ACTION TICKETS")
print("="*50 + "\n")

mbu_tickets = generate_mbu_compliance_tickets(df_enrol, df_bio)
fraud_tickets = generate_fraud_alert_tickets(df_results)
migration_tickets = generate_migration_hotspot_tickets(df_enrol, df_demo)

all_tickets = mbu_tickets + fraud_tickets + migration_tickets

print(f"\n✅ Total tickets generated: {len(all_tickets):,}")

In [ ]:
# View sample tickets
print("📋 Sample Tickets:")
pd.DataFrame(all_tickets[:5])

---
## 💾 Part 4: Generate Output JSON

In [ ]:
# Generate compliance map data
def generate_compliance_map_data(df_enrol: pd.DataFrame, df_bio: pd.DataFrame) -> List[Dict[str, Any]]:
    df_merged = df_enrol[['pincode', 'district', 'state', 'age_5_17']].merge(
        df_bio[['pincode', 'bio_age_5_17']], on='pincode', how='left'
    )
    df_merged['bio_age_5_17'] = df_merged['bio_age_5_17'].fillna(0)
    df_merged['deficit'] = df_merged['age_5_17'] - df_merged['bio_age_5_17']
    
    df_positive = df_merged[df_merged['deficit'] > 0]
    
    return [{
        'pincode': row['pincode'],
        'district': row['district'],
        'state': row['state'],
        'deficit': int(row['deficit'])
    } for _, row in df_positive.iterrows()]

compliance_map_data = generate_compliance_map_data(df_enrol, df_bio)
print(f"✓ Generated compliance data for {len(compliance_map_data):,} pincodes")

In [ ]:
# Calculate summary statistics
total_pincodes = len(df_enrol)
critical_alerts = sum(1 for t in all_tickets if t['priority'] == 'Critical')

# MBU backlog total
df_merged = df_enrol[['pincode', 'age_5_17']].merge(
    df_bio[['pincode', 'bio_age_5_17']], on='pincode', how='left'
)
df_merged['bio_age_5_17'] = df_merged['bio_age_5_17'].fillna(0)
df_merged['deficit'] = df_merged['age_5_17'] - df_merged['bio_age_5_17']
mbu_backlog_total = int(df_merged[df_merged['deficit'] > 0]['deficit'].sum())

# Prepare action tickets
priority_order = {'Critical': 0, 'High': 1, 'Medium': 2, 'Low': 3}
action_tickets = [{
    'pincode': t['pincode'],
    'priority': t['priority'],
    'task': t['task'],
    'venue': t['venue'],
    'details': t['details'],
    'whatsapp_msg': t['whatsapp_msg']
} for t in all_tickets]
action_tickets.sort(key=lambda x: priority_order.get(x['priority'], 99))

# Final output structure
output = {
    'summary': {
        'total_pincodes': total_pincodes,
        'critical_alerts': critical_alerts,
        'mbu_backlog_total': mbu_backlog_total,
        'high_priority_tickets': sum(1 for t in all_tickets if t['priority'] in ['Critical', 'High']),
        'total_tickets': len(all_tickets)
    },
    'compliance_map_data': compliance_map_data,
    'action_tickets': action_tickets
}

# Add anomaly summary
if 'is_high_risk' in df_results.columns:
    output['anomaly_summary'] = {
        'total_analyzed': len(df_results),
        'high_risk_count': int(df_results['is_high_risk'].sum()),
        'average_confidence': float(df_results[df_results['is_high_risk']]['confidence_score'].mean()) if df_results['is_high_risk'].any() else 0
    }

print("✅ Output structure created!")
print(f"\n📊 Summary:")
print(f"   Total Pincodes: {output['summary']['total_pincodes']:,}")
print(f"   Critical Alerts: {output['summary']['critical_alerts']:,}")
print(f"   MBU Backlog: {output['summary']['mbu_backlog_total']:,}")
print(f"   Total Tickets: {output['summary']['total_tickets']:,}")

In [ ]:
# Save to JSON
output_path = Path("./app_data.json")

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

print(f"\n✅ Output saved to: {output_path.absolute()}")

---
## ✅ Complete!

The `app_data.json` file has been generated and is ready for the frontend dashboard.